# 🎬 Video Trans Studio - AI 视频配音与口型同步
### 核心能力：
- **ASR**: Faster-Whisper (Large-v3)
- **Translation**: NLLB-200 (HuggingFace Local)
- **TTS**: Index-TTS2 (Zero-shot Voice Cloning)
- **LipSync**: Wav2Lip-GAN

> **运行环境建议**：Google Colab Tesla T4 (16GB VRAM)

In [1]:
# @title 🔍 0. 环境检查 (排查问题专用)
import sys
import os
import torch
import platform
import psutil
from IPython.display import HTML, display

def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor: return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

print("="*50)
print("💻 系统硬件环境")
print("="*50)
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"处理器: {platform.processor()}")
print(f"物理核心数: {psutil.cpu_count(logical=False)}")
print(f"逻辑核心数: {psutil.cpu_count(logical=True)}")
print(f"总内存: {get_size(psutil.virtual_memory().total)}")

print("\n" + "="*50)
print("🚀 显卡 (GPU) 环境")
print("="*50)
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        prop = torch.cuda.get_device_properties(i)
        print(f"GPU {i}: {prop.name}")
        print(f"  - 显存总量: {prop.total_memory / 1024**3:.2f} GB")
        print(f"  - 计算能力: {prop.major}.{prop.minor}")
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
else:
    print("❌ 未检测到可用 GPU，请在 '修改' -> '笔记本设置' 中开启 T4 GPU 加速。")

print("\n" + "="*50)
print("🐍 软件开发环境")
print("="*50)
print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"当前路径: {os.getcwd()}")

!nvidia-smi

💻 系统硬件环境
操作系统: Linux 6.6.105+
处理器: x86_64
物理核心数: 1
逻辑核心数: 2
总内存: 12.67GB

🚀 显卡 (GPU) 环境
GPU 0: Tesla T4
  - 显存总量: 14.74 GB
  - 计算能力: 7.5
CUDA 版本: 12.6
cuDNN 版本: 91002

🐍 软件开发环境
Python 版本: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch 版本: 2.9.0+cu126
当前路径: /content
Sat Jan 31 07:21:42 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   

In [2]:
# @title 🚀 1. 环境初始化
import os
import sys

# 1. 确保在 /content 目录下操作
os.chdir('/content')

# 2. 克隆项目并进入
if not os.path.exists('video-trans-studio'):
    !git clone https://github.com/infinite-gaming-studio/video-trans-studio.git

%cd video-trans-studio
!git fetch --all
!git reset --hard origin/main
!git pull

# 3. 智能环境检测与安装

if not os.path.exists(env_marker):
    print("⏳ 正在进行首次环境安装 (大约 3-5 分钟)...")
    !bash setup_colab.sh
    with open(env_marker, 'w') as f: f.write('done')
    
    print("\n" + "!"*50)
    print("⚠️ 环境安装已完成！")
    print("⚠️ 由于 Colab 机制，请务必手动点击上方菜单栏: ")
    print("   '运行时' -> '重新启动会话' (Runtime -> Restart Session)")
    print("⚠️ 重启后，直接从 '第 2 步' 开始运行即可。")
    print("!"*50)
else:
    try:
        import transformers
        print("✅ 环境已就绪，Transformers 版本:", transformers.__version__)
    except ImportError:
        print("❌ 环境虽然已安装，但尚未重启会话，请点击 '运行时' -> '重新启动会话'。")

/content/video-trans-studio
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 600 bytes | 600.00 KiB/s, done.
From https://github.com/infinite-gaming-studio/video-trans-studio
   7db00da..61d8862  main       -> origin/main
Updating 7db00da..61d8862
Fast-forward
 core/lipsync.py | 12 ++++++++++--
 1 file changed, 10 insertions(+), 2 deletions(-)
✅ 环境已就绪，Transformers 版本: 4.38.0


In [3]:
# @title 📂 2. 选择素材
import os
import glob
from pathlib import Path

# 核心逻辑：自动抓取服务器根目录下 sample_video 文件夹中最新上传的视频
# 在 Colab 中，根目录通常是 /content
colab_root = Path('/content')
sample_dir = colab_root / 'sample_video'

# 兼容性处理：如果不在 Colab 环境，尝试查找当前目录下的 sample_video
if not sample_dir.exists():
    sample_dir = Path('sample_video')

video_path = None

if sample_dir.exists():
    # 支持的视频格式
    valid_extensions = ['*.mp4', '*.mov', '*.avi', '*.mkv', '*.webm']
    video_files = []
    for ext in valid_extensions:
        video_files.extend(glob.glob(str(sample_dir / ext)))
    
    if video_files:
        # 按修改时间（mtime）排序，取最新的一个
        video_path = max(video_files, key=os.path.getmtime)
        print(f"✅ 成功定位服务器根目录素材: {video_path}")
        print(f"🕒 最新上传时间: {os.path.getmtime(video_path)}")
    else:
        print(f"❌ 在 {sample_dir} 文件夹中未找到任何视频文件。")
else:
    print(f"❌ 未找到目录: {sample_dir.absolute()}")
    print("💡 请确保在 Colab 左侧侧边栏的根目录（/content）下创建了 sample_video 文件夹。")

✅ 成功定位服务器根目录素材: /content/sample_video/bilibili_BV1ZzzDBKEhF_852x480.mp4
🕒 最新上传时间: 1769838529.7385452


In [4]:
# @title ⚙️ 3. 运行全自动流水线
target_language = "en" # @param ["zh-cn", "en", "es", "fr", "ja"]
use_local_translation = True # @param {type:"boolean"}

import sys
import torch
import importlib
import os

# 确保在项目目录中运行
if os.getcwd() != '/content/video-trans-studio':
    %cd /content/video-trans-studio

# 强制重载自定义模块，防止代码缓存 (First Principles: Memory vs Disk Sync)
# 这一步至关重要，因为 Notebook 会缓存 import 的模块
modules_to_reload = ['main', 'config', 'core.tts', 'core.lipsync', 'core.utils', 'core.asr', 'core.audio', 'core.translator']
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

# 检查核心依赖，如果报错则提供修复方案
try:
    from transformers import pipeline
    from main import run_pipeline
except ImportError as e:
    print(f"❌ 模块加载失败: {e}")
    print("\n🔄 尝试自动紧急修复环境...")
    !pip install transformers==4.38.0 --force-reinstall
    print("⚠️ 环境已重置，请务必点击上方 '运行时' -> '重新启动会话'，然后再次运行此单元格。")
    sys.exit()

# 强制清理显存并执行
torch.cuda.empty_cache()

if 'video_path' in locals() and video_path:
    # 使用 print 实时刷新输出
    print(f"🎬 开始处理视频: {video_path}")
    # 顶层 await 调用异步函数
    await run_pipeline(video_path, target_lang=target_language)
else:
    print("❌ 错误：未定义 video_path，请先成功运行 '第 2 步'。")

print("\n✨ 处理全流程结束！")

🎬 开始处理视频: /content/sample_video/bilibili_BV1ZzzDBKEhF_852x480.mp4
✅ Running on: CUDA
🚀 GPU: Tesla T4
💾 VRAM: 15.83 GB
📂 Output Dir: /content/video-trans-studio/output
⏳ [PROGRESS] |--------------------| 0% | STEP: Initializing (Waiting) | Step Time: 0.0s | Total: 0.0s🎬 Extracting audio from /content/sample_video/bilibili_BV1ZzzDBKEhF_852x480.mp4...
⏳ [PROGRESS] |--------------------| 0% | STEP: Audio Extraction (Extracting Wav) | Step Time: 5.0s | Total: 5.0s✅ Audio extracted to: /content/video-trans-studio/temp/original_audio.wav
⏳ Loading Whisper Model (large-v3)...
⏳ [PROGRESS] |████----------------| 20% | STEP: ASR Transcription (Whisper Large-v3) | Step Time: 9.3s | Total: 15.0s

Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(



⏳ [PROGRESS] |████----------------| 20% | STEP: ASR Transcription (Whisper Large-v3) | Step Time: 13.3s | Total: 19.0s✅ Whisper Model Loaded.
🎙️ Transcribing: /content/video-trans-studio/temp/original_audio.wav...
⏳ [PROGRESS] |████----------------| 20% | STEP: ASR Transcription (Whisper Large-v3) | Step Time: 103.5s | Total: 109.2s✅ Transcription complete. Detected language: zh
🗑️ Whisper Model Unloaded.
🌍 Translating 324 segments...
⏳ [PROGRESS] |████████------------| 40% | STEP: Translation (NLLB to en) | Step Time: 25.1s | Total: 135.3s🗣️ Generating TTS audio for 324 segments via Edge-TTS...
⏳ Downloading segments in parallel...
⏳ [PROGRESS] |████████████--------| 60% | STEP: TTS Generation (Edge-TTS Generating) | Step Time: 21.3s | Total: 157.3s🧩 Combining audio segments with proper timing...
⏳ [PROGRESS] |████████████--------| 60% | STEP: TTS Generation (Edge-TTS Generating) | Step Time: 83.4s | Total: 219.5s✅ Audio generated: /content/video-trans-studio/output/bilibili_BV1ZzzDBK

In [ ]:
# @title 📺 4. 预览结果
import os
import glob
from IPython.display import Video

output_dir = 'output'
# 递归查找所有以 final_ 开头的 mp4 文件
final_videos = glob.glob(os.path.join(output_dir, '**', 'final_*.mp4'), recursive=True)

if final_videos:
    # 按照文件修改时间排序，展示最新生成的一个
    latest_video = max(final_videos, key=os.path.getmtime)
    print(f"🎬 展示最新生成的视频: {latest_video}")
    display(Video(latest_video, embed=True, width=600))
else:
    print("❌ 未找到生成的视频文件。")